In [1]:
# from bs4 import BeautifulSoup
import requests
# from selenium import webdriver
from langchain.schema import Document
import jsonlines
import re
import json

In [6]:
# only letters and numbers
LETTERS_NUMBERS_PATTERN = re.compile("[^a-zA-Z0-9ąćęłńóśźżĄĆĘŁŃÓŚŹŻ]+")

# only letters
LETTERS_PATTERN = re.compile("[^a-zA-ZąćęłńóśźżĄĆĘŁŃÓŚŹŻ]+")

NOMINATIM_API = "https://nominatim.openstreetmap.org/search?q={query}&format=json&addressdetails=1&limit=1"
NOMINATIM_HEADERS = {"User-Agent": "GeoSpatialLLM/1.0"}

BOOKSY_URL = "https://booksy.com/pl-pl/s/13750_wroclaw?businessesPage={page_num}"
BOOKSY_BASE_URL = "https://booksy.com"

In [ ]:
# options = webdriver.ChromeOptions()
# options.add_argument("--headless")
# driver = webdriver.Chrome(options=options)

In [ ]:
data = []
idx = 0

for page_num in range(1, 160):

    try:
        page_url = BOOKSY_URL.format(page_num=page_num)
        response = requests.get(page_url)
        bs = BeautifulSoup(response.text, "html.parser")

        businesses = bs.find_all(
            class_="purify_jFNrjJsLiLW+6sHFuYDiWA== purify_gvbw+GpawACpHIzUwLBo8A== purify_CA-Z3uLarGK6crMd3kMgRw=="
        )

        for business in businesses:
            business_dict = {}
            business_dict["id"] = idx

            name = bs.find(class_="purify_3UeKw5MV+NerIrQNs8HSXA== purify_IbX1Bb-sl2ffhqyi9xUH1A==")
            clean_name = name.text.replace("\n", "").replace("  ", " ").strip()
            business_dict["name"] = clean_name

            href = business.find("a")["href"]
            business_url = BOOKSY_BASE_URL + href
            response = requests.get(business_url)
            bs = BeautifulSoup(response.text, "html.parser")

            address_parent_div = (
                bs.find(
                    class_="purify_prm7MfDXczhTZvcY5KwOuA== purify_Sardy6hfiet162IZ2pYFPA== purify_m9mNOPjpHD0tNTW6GC+hEw=="
                )
                .text.replace("\n", "")
                .strip()
            )
            address = address_parent_div.split("div")[0].replace("\n", "").strip()
            business_dict["address"] = address

            rating = business.find(
                class_="purify_aVm421MES2COps1la9BuOQ== purify_3dZ+zycQyUtqfDxWbTGTlw== purify_kq4BZr36QXoLgkAnN95TWw=="
            ).text.strip()
            business_dict["rating"] = rating

            services = bs.find_all(class_="purify_TDNkQVqTWE1e9+GIIAxpIA==")
            prices = bs.find_all(
                class_="purify_s8fylQf1Z9K4N9kwPLHJaA== purify_IbX1Bb-sl2ffhqyi9xUH1A== purify_f9tJlQGTtqZe8Hg0f7oQhg=="
            )
            business_dict["services"] = []
            for service, price in zip(services, prices):
                try:
                    service_dict = {}
                    service_clean = (
                        LETTERS_PATTERN.sub(" ", service.text)
                        .replace("\n", "")
                        .replace("  ", " ")
                        .strip()
                        .lower()
                    )
                    price_clean = (
                        price.text.replace("\n", "").replace("  ", " ").replace("\xa0", " ").strip()
                    )

                    service_dict["price"] = price_clean
                    service_dict["name"] = service_clean
                    business_dict["services"].append(service_dict)
                except Exception as e:
                    print(f"Error: {e}")
                    continue

            idx += 1

            # try:
            #     driver.get(page_url)
            #     html = driver.page_source
            #     soup = BeautifulSoup(html, "html.parser")
            #     soup_str = str(soup)
            #     description = soup_str[
            #         soup_str.find("description:") : soup_str.find(",business_categories")
            #     ]
            #     business_dict["description"] = description
            # except Exception as e:
            #     print(f"Error: {e}")
            #     business_dict["description"] = None

            data.append(business_dict)

    except Exception as e:
        print(f"Error: {e}")
        continue

In [2]:
def write_json(data, path: str = "./data/booksy_wroclaw.json"):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)


def open_json(file: str):
    with open(file, "r", encoding="utf-8") as f:
        return json.load(f)


def json_to_jsonlines(path: str):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
        with jsonlines.open(path.replace(".json", ".jsonlines"), "w") as writer:
            writer.write_all(data)

In [3]:
# write_json(data)
# json_to_jsonlines("../data/booksy_wroclaw.json")
data = open_json("../data/booksy_wroclaw_geocoded.json")

In [73]:
def extract_address_components(address):
    pattern = r"(?P<street>(?:ul\.)?\s*[a-ząćęłńóśźżA-ZĄĆĘŁŃÓŚŹŻ\s]+?),?\s+(?P<number>\d+[a-zA-Z]?[-/\d\sa-zA-Z]*)?,?.*?(?P<city>[A-ZŁŚŻŹĆĄĘÓ][a-ząćęłńóśźżA-ZĄĆĘŁŃÓŚŹŻ]*)?"

    phrases_to_remove = [
        "ulica",
        "ul.",
        "ul",
    ]

    pattern_prefix = re.compile(r"|".join(phrases_to_remove), re.IGNORECASE)
    address = pattern_prefix.sub("", address)

    fail_flag = False

    match = re.search(pattern, address)
    try:
        street = (match.group("street").strip() or "").strip()
    except:
        print(f"Failed to extract street from address: {address}")
        fail_flag = True
        street = ""

    try:
        number = (match.group("number") or "").strip()
    except:
        print(f"Failed to extract number from address: {address}")
        number = ""

    try:
        city = (match.group("city") or "Wrocław").strip()
    except:
        print(f"Failed to extract city from address: {address}")
        if fail_flag:
            return None
        city = "Wrocław"

    return f"{street}+{number}+{city}+Wroclaw"


for business in data:
    new_address = extract_address_components(business["address"])
    if new_address:
        business["address_clean"] = new_address
    else:
        business["address_clean"] = None

Failed to extract street from address: Łęczycka,20, 53-632, Wrocław
Failed to extract number from address: Łęczycka,20, 53-632, Wrocław
Failed to extract city from address: Łęczycka,20, 53-632, Wrocław
Failed to extract street from address: 50-075, Wrocław
Failed to extract number from address: 50-075, Wrocław
Failed to extract city from address: 50-075, Wrocław
Failed to extract street from address: 50-075, Wrocław
Failed to extract number from address: 50-075, Wrocław
Failed to extract city from address: 50-075, Wrocław
Failed to extract street from address: 50-101, Wrocław
Failed to extract number from address: 50-101, Wrocław
Failed to extract city from address: 50-101, Wrocław
Failed to extract street from address: 55-040, Kobierzyce
Failed to extract number from address: 55-040, Kobierzyce
Failed to extract city from address: 55-040, Kobierzyce


In [37]:
for business in data:
    try:
        address_clean = business["address_clean"]
        address_clean = address_clean.replace(" ", "+")
        address_clean = address_clean.replace("++", "+")
        address_clean = address_clean.replace("lokal", "")
        address_clean = address_clean.replace("lok", "")
        business["address_clean"] = address_clean
    except:
        print(business)

write_json(data, "../data/booksy_wroclaw_clean.json")

In [ ]:
def reverese_geocod(address):
    url = NOMINATIM_API.format(query=address)
    response = requests.get(url, headers=NOMINATIM_HEADERS)
    response.raise_for_status()
    return response.json()

In [ ]:
errors = []

for business in data:
    if business["address_clean"]:
        try:
            response = reverese_geocod(business["address_clean"])
            business["geolocation"] = {}
            business["geolocation"]["latitude"] = response[0]["lat"]
            business["geolocation"]["longitude"] = response[0]["lon"]
            business["geolocation"]["place_id"] = response[0]["place_id"]
            business["geolocation"]["bbox"] = response[0]["boundingbox"]
        except Exception as e:
            print(f"Error for business: {business['name']} with ID {business['id']}, {e}")
            business["geolocation"] = None
            errors.append(business)
    else:
        business["geolocation"] = None
        errors.append(business)

In [ ]:
for business in data:
    for service in business["services"]:
        price = service["price"]
        if not any(char.isdigit() for char in price):
            service["price"] = 0
        else:
            service["price"] = float(service["price"].replace(",", ".").split(" ")[0])
        # print(f"{price} -> {service['price']}")

In [4]:
for business in data:
    business["rating"] = float(business["rating"].replace(",", "."))

write_json(data, "../data/booksy_wroclaw_geocoded.json")